In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re 
from tqdm import tqdm_notebook as tqdm

In [28]:
#データフレームの扱い。カラムを増やす。
pd.options.display.notebook_repr_html = True
pd.options.display.max_colwidth = 1000 #最大文字数の変更
pd.options.display.max_rows = 1000 #最大行数の変更

In [16]:
#J1各チームの日程・結果にアクセスするためのurlリスト。
html_all = urlopen("https://soccer.yahoo.co.jp/jleague/teams/j1")
soup_all = BeautifulSoup(html_all, "html.parser")
team_url = soup_all.find_all(href = re.compile("^/jleague/teams/schedule"))
team_url_list = []
team_name_list= []
for a in tqdm(team_url):
    team_url_list.append("https://soccer.yahoo.co.jp" + a.get("href"))
    html = urlopen("https://soccer.yahoo.co.jp" + a.get("href"))
    soup = BeautifulSoup(html, "html.parser")
    team_name = str(soup.find_all(class_ = "name"))
    team_name = re.search(pattern, team_name)
    team_name_list.append(team_name.group()[1:-1])
print(*team_name_list)


北海道コンサドーレ札幌 ベガルタ仙台 鹿島アントラーズ 浦和レッズ FC東京 川崎フロンターレ 横浜F・マリノス 湘南ベルマーレ 松本山雅FC 清水エスパルス ジュビロ磐田 名古屋グランパス ガンバ大阪 セレッソ大阪 ヴィッセル神戸 サンフレッチェ広島 サガン鳥栖 大分トリニータ


In [6]:
def score(soup):
    #各試合のスコアのTag
    score_tag = soup.find_all(href =  re.compile("^/jleague/game/score/"))
    #Tagからスコアをget
    pattern = "\d-\d"
    score_list = str(score_tag).replace("\xa0","")
    score_list = re.findall(pattern, score_list)
    return(score_tag, score_list)

In [17]:
#結果を格納しているタグを発見してテキストにする。
def results(soup):
    results = soup.find_all(class_="textC")
    #勝敗だけ抽出。
    results_list = []
    for v in results:
        if (v.text == "△" or v.text == "●" or v.text == "○" ):
            results_list.append(v.text)
    #リストの勝敗を数値に変換。
    results_list = ["2" if "△" in s  else s for s in results_list]
    results_list = ["1" if "○" in s  else s for s in results_list]
    results_list = ["0" if "●" in s  else s for s in results_list]
    return(results_list)

In [18]:
def home_review(score_tag, teamname):
    #Home/Away
    home_away_url = []
    home_away_list = []
    #review
    match_review = []
    #試合詳細のURLをリスト化
    for a in score_tag:
        home_away_url.append("https://soccer.yahoo.co.jp"  + a.get("href"))
    #リスト化されたURLからHOME/AWAY情報を抽出
    for i in tqdm(home_away_url, teamname):
        home_away_html = urlopen(i)
        home_away_soup = BeautifulSoup(home_away_html, "html.parser")
        if str(home_away_soup.title.contents[0]).split()[-7] == teamname :
            home_away_list.append(1)
        else:
            home_away_list.append(0)
        review_test = home_away_soup.find("p",class_="", text=re.compile("。")).text
        pattern = re.compile("掲載")
        matchObj = pattern.search(review_test)
    #掲載　という文字が入っていたらレビューではないので空にする。
        if matchObj:
            match_review.append("Nah")
        else:
            match_review.append(review_test)
    return(home_away_url, home_away_list, match_review)

In [51]:
###xに入るのは数字
###team x の情報を集める関数。
def gather_information(x):
    #各チームにアクセス。
    html = urlopen(team_url_list[x])
    soup = BeautifulSoup(html, "html.parser")
    #チーム名
    teamname = team_name_list[x]   
    df_score = score(soup)
    score_tag = df_score[0]
    df_results = results(soup)
    df_home_review = home_review(score_tag, teamname)
    #DataFrame→CSV にして出力
    df = pd.DataFrame({"results" : df_results,
                                     "home" : df_home_review[1],
                                     "score" : df_score[1],
                                       "match review" : df_home_review[2]
                                  })
    df.to_csv("/Users/yamadaikuya/gitrepos/toto/trial/"+str(x)+".csv",sep=",")

In [52]:
gather_information(0)

In [53]:
#全てのチームに関して情報を集める。
def gather_all_information():
    for i in tqdm(range(len(team_url_list)),"Overall Progress"):
        gather_information(i)

In [54]:
gather_all_information()

In [65]:
#csvからdfに戻す。
def show_dataframe(i):
    df_return = pd.read_csv("/Users/yamadaikuya/gitrepos/toto/trial/"+str(i)+".csv", 
                      index_col=[0])
    df_return_latest = df_return[-1:] #最新のが見たいときはこっち
    return(display(df_return))

In [66]:
print(team_name_list[0])
df_return = pd.read_csv("/Users/yamadaikuya/gitrepos/toto/trial/0.csv", 
                  index_col=[0])
df_return

北海道コンサドーレ札幌


,results,home,score,match review
0,0,0,2-0,昨季飛躍を果たした両チームの対戦はホームの湘南に軍配が上がった。ボールを支配する札幌に押し込まれる展開が続いたが、それでも運動量で相手を圧倒する「湘南スタイル」は今季も健在。時間帯が深まるにつれて徐々に流れを引き寄せると、「復帰戦」の武富が気迫のこもった2得点を挙げる。BMWスを歓喜の渦に巻き込み、3年連続の開幕戦勝利に大きく貢献した。対する札幌は攻撃陣がかみ合わず、無念の完封負け。前線ではチャナティップの単騎突破に頼るシーンが目立ち、Aロペスや鈴木ら新戦力の一刻も早いフィットが求められる。
1,1,0,0-2,開始早々に出はなをくじかれた浦和。これで相手に勢いと自信を与えてしまった。ボールを持った際は札幌の素早いプレスに自由を奪われ、またチャナティップを中心とした華麗なパスワークで何度もゴールを脅かされる展開に。シュートミスや西川のセーブに救われて早い時間帯での大量失点は回避したものの、後半も大勢は変わらず。終盤は山中や途中出場のAナバウトの突破から波状攻撃を見せるも、攻撃のスイッチを入れるにはあまりにも遅すぎた。結果、なすすべもなく0－2で完敗し、かつての恩師に勝点3の手土産を渡してしまった。
2,2,0,1-1,Nah
3,1,1,5-2,前節にアウェイで今季初勝利を飾り、ホーム開幕戦を迎えた札幌。守備時には全員が自陣に戻ってボールを奪取し、攻撃ではAロペスを中心に左右にボールを展開。試合を優位に進め、1点リードで折り返す。後半は開始直後から前がかりになった清水に対し、今度はカウンター攻撃で応戦。これで勢いはさらに増し、素早い攻撃から何度もゴールへと迫る。終わってみればAロペスの連続追撃弾で大量5得点を挙げての白星を飾った。一方の清水は、開幕からの3戦で10失点を喫しており、守備の改善が求められる。
4,2,1,0-0,Nah
5,0,1,1-3,清水戦では、鈴木とAロペスの前線2人で5得点を奪った札幌だが、今節の相手は11試合連続勝ちなしと相性の悪い鹿島。積極的な攻撃を展開するが、「試合巧者な鹿島」に2点リードを許し、試合を折り返す。後半開始から徐々に攻撃のリズムを取り戻し、押し込む時間帯が続くも、レアンドロに3点目を奪われてしまう。後半40分にAロペスの2試合連続ゴールで1点を返すが、反撃もここまで。今回も苦汁をなめさせられる結果となった。
6,0,0,4-0,4節での敗戦から8連敗を喫した昨季との違いを見せることに成功した名古屋。速い出足で中盤を支配するほか、パスワークの際の味方との関わり方で相手を圧倒すると、決定機をことごとく沈めて前半だけで3点を奪って折り返す。後半に入っても集中を切らさずに1点を追加する。すると、守備面でも米本を中心に札幌のカウンターをつぶすなど、光るプレーを連発。最後まで統率されたラインで相手の攻撃を抑え、ほぼ完璧な試合運びで今季三度目の完封勝利を達成した。
7,0,1,1-2,前からの激しいプレスと多くの味方の攻撃参加でゴールを目指す札幌と、後方からつなぎつつ、有効なスペースを突いて攻め込む大分。両者のスタイルのぶつかり合いが終始続いた一戦となった。札幌は開始早々に先手を取られると、プレスの意識をさらに強化。だが、これで裏のスペースを空けると、次第に藤本らに突かれて2点目を献上する。大分としては狙い通りの、札幌としては勢いに水を差される1点となった。後半も試合の大勢はほぼ変わらず。ホームチームは90分にわたって攻撃に圧力を見せ続けただけに、悔しい敗戦となった。
8,1,1,4-1,Nah
9,1,0,0-1,両チームともに締まったプレーを見せ、緊張感のある固い内容の試合となる。ホームのC大阪は連係でペナルティエリアへの進入を試みる札幌の攻撃に耐え、仕掛ける際には前への意識が高い片山や松田を中心に右サイドからの打開を狙う。後半に入ってもなお、一進一退の攻防が続くが、セットプレーから進藤に今季初得点を決められ、苦しい展開になる。その後は収まりの良い都倉に縦パスを入れて攻撃を仕掛け続けるも、タイムアップ。復調のけはいを見せていただけに、痛い敗戦を喫してしまった。


In [67]:
for i in range(len(team_name_list)):
    show_dataframe(i)

,results,home,score,match review
0,0,0,2-0,昨季飛躍を果たした両チームの対戦はホームの湘南に軍配が上がった。ボールを支配する札幌に押し込まれる展開が続いたが、それでも運動量で相手を圧倒する「湘南スタイル」は今季も健在。時間帯が深まるにつれて徐々に流れを引き寄せると、「復帰戦」の武富が気迫のこもった2得点を挙げる。BMWスを歓喜の渦に巻き込み、3年連続の開幕戦勝利に大きく貢献した。対する札幌は攻撃陣がかみ合わず、無念の完封負け。前線ではチャナティップの単騎突破に頼るシーンが目立ち、Aロペスや鈴木ら新戦力の一刻も早いフィットが求められる。
1,1,0,0-2,開始早々に出はなをくじかれた浦和。これで相手に勢いと自信を与えてしまった。ボールを持った際は札幌の素早いプレスに自由を奪われ、またチャナティップを中心とした華麗なパスワークで何度もゴールを脅かされる展開に。シュートミスや西川のセーブに救われて早い時間帯での大量失点は回避したものの、後半も大勢は変わらず。終盤は山中や途中出場のAナバウトの突破から波状攻撃を見せるも、攻撃のスイッチを入れるにはあまりにも遅すぎた。結果、なすすべもなく0－2で完敗し、かつての恩師に勝点3の手土産を渡してしまった。
2,2,0,1-1,Nah
3,1,1,5-2,前節にアウェイで今季初勝利を飾り、ホーム開幕戦を迎えた札幌。守備時には全員が自陣に戻ってボールを奪取し、攻撃ではAロペスを中心に左右にボールを展開。試合を優位に進め、1点リードで折り返す。後半は開始直後から前がかりになった清水に対し、今度はカウンター攻撃で応戦。これで勢いはさらに増し、素早い攻撃から何度もゴールへと迫る。終わってみればAロペスの連続追撃弾で大量5得点を挙げての白星を飾った。一方の清水は、開幕からの3戦で10失点を喫しており、守備の改善が求められる。
4,2,1,0-0,Nah
5,0,1,1-3,清水戦では、鈴木とAロペスの前線2人で5得点を奪った札幌だが、今節の相手は11試合連続勝ちなしと相性の悪い鹿島。積極的な攻撃を展開するが、「試合巧者な鹿島」に2点リードを許し、試合を折り返す。後半開始から徐々に攻撃のリズムを取り戻し、押し込む時間帯が続くも、レアンドロに3点目を奪われてしまう。後半40分にAロペスの2試合連続ゴールで1点を返すが、反撃もここまで。今回も苦汁をなめさせられる結果となった。
6,0,0,4-0,4節での敗戦から8連敗を喫した昨季との違いを見せることに成功した名古屋。速い出足で中盤を支配するほか、パスワークの際の味方との関わり方で相手を圧倒すると、決定機をことごとく沈めて前半だけで3点を奪って折り返す。後半に入っても集中を切らさずに1点を追加する。すると、守備面でも米本を中心に札幌のカウンターをつぶすなど、光るプレーを連発。最後まで統率されたラインで相手の攻撃を抑え、ほぼ完璧な試合運びで今季三度目の完封勝利を達成した。
7,0,1,1-2,前からの激しいプレスと多くの味方の攻撃参加でゴールを目指す札幌と、後方からつなぎつつ、有効なスペースを突いて攻め込む大分。両者のスタイルのぶつかり合いが終始続いた一戦となった。札幌は開始早々に先手を取られると、プレスの意識をさらに強化。だが、これで裏のスペースを空けると、次第に藤本らに突かれて2点目を献上する。大分としては狙い通りの、札幌としては勢いに水を差される1点となった。後半も試合の大勢はほぼ変わらず。ホームチームは90分にわたって攻撃に圧力を見せ続けただけに、悔しい敗戦となった。
8,1,1,4-1,Nah
9,1,0,0-1,両チームともに締まったプレーを見せ、緊張感のある固い内容の試合となる。ホームのC大阪は連係でペナルティエリアへの進入を試みる札幌の攻撃に耐え、仕掛ける際には前への意識が高い片山や松田を中心に右サイドからの打開を狙う。後半に入ってもなお、一進一退の攻防が続くが、セットプレーから進藤に今季初得点を決められ、苦しい展開になる。その後は収まりの良い都倉に縦パスを入れて攻撃を仕掛け続けるも、タイムアップ。復調のけはいを見せていただけに、痛い敗戦を喫してしまった。


,results,home,score,match review
0,2,1,0-0,互いに不完全燃焼のまま90分を終えた。仙台は右サイドを主軸に据えて攻め込む。蜂須賀とハモンロペスは敵陣で多くボールに触れ、長沢のフィニッシュシーンを演出するが、攻め上がる回数が少ないこともあってネットを揺らすまでには至らない。逆に相手のパスワークに振り回され、守備に追われる時間を長く過ごす展開に。それでも、集中力を切らさずに体を張り続け、無失点を達成したことは収穫か。一方、浦和はボールこそ運べるものの、枠をとらえる強いシュートは数少なく、いまだに「調整段階」であることを印象づける結果となった。
1,0,0,2-1,序盤から、細かくパスをつないで敵陣に押し込む横浜FMと、低く引いて構えつつ長沢に徹底的に放り込む仙台、という構図が続く。ホームチームはPKから先手を取ると、崩しのスピードと流動性がさらに向上。中央で相手の注意を引き付け、その間に別の味方が大外へ開いてフリーでボールを呼び込む、という攻撃の形から何度もチャンスを迎える。この策から2点目を奪った後も三好や仲川を中心に攻め立て、相手に休む暇を与えない。終盤にPKを献上したものの、横浜FMは90分間主導権を握り続けて「得意先」から危なげなく勝点3を得た。
2,1,0,1-3,Nah
3,0,1,1-3,「特別な一戦で勝利を」という大きな決意を胸に臨んだホームの仙台。手数を掛けない高速カウンターで何度もゴールに迫り、リードを奪った立ち上がりの試合運びにその強い意気込みを感じたサポーターも多いだろう。だが、前半のうちに追い付かれ、さらに後半早々に出はなをくじかれた後は中盤で安易なミスを連発。手渡されたチャンスを「VIPトリオ」が逃すはずもなく、結果的に3失点を喫して敗戦。石原崇や吉尾ら新加入選手が存在感を発揮した点は収穫だが、守備面で大きな課題を残し、リーグ戦初白星は次節以降に持ち越しとなった。
4,1,1,2-1,Nah
5,0,0,2-1,主力を大幅に欠いた湘南だが、実戦経験の少ない若手も伸び伸びとプレーし、違和感なくいつもの「湘南スタイル」を披露。また、相手が終盤まで足が止まっていたため、「急造メンバー」ながら敵陣に押し込み続けて終始主導権を握る。流れの中でDF2人に生まれたゴールも、決して偶然の産物ではないだろう。チームとしての総合力の高さを示した。終盤は選手交代で攻撃の形をつかんだ仙台の波状攻撃を受けるが、堅実な守備に徹して2－1で終了。一方の仙台は終盤を迎えるまでほとんど相手に脅威を与えられず。最下位となってしまった。
6,0,1,0-2,3連敗中のチーム同士の対戦は戦術の浸透度が勝負を決めた。開始直後こそ前線の3枚が掛けるプレッシングがはまった仙台だったが、徐々に相手の優れたパスワークに対して後手に回るようになり、中盤を制圧されてしまう。後半からは2シャドーのハモンロペスと吉尾のポジションを左右で入れ替えたが、コンビネーションプレーはあまり見られなかった。一方のC大阪は時間がたつにつれて選手の距離感が良くなり、奥埜とソウザが攻守に渡って奮闘。また、今季初出場の田中にゴールが生まれ、雪が降りしきるユアスタで快勝を収めた。
7,1,1,3-0,横浜FMから加入した若武者がユアスタに今季初めての歓喜をもたらした。2戦連続の先発出場を果たした吉尾は、序盤から積極的な姿勢で攻撃に関与する。まず、輝きを放ったのは前半15分。巧みな動き出しで味方からのパスを引き出すと、右サイドから左足で正確なクロスを蹴り込み、ジャーメインの先制点をお膳立て。豊富な運動量でピッチを駆け回るレフティーは、後半9分にも得点に絡む。相手のパスミスに反応してシュートを放ち、ポストの跳ね返りを兵藤が押し込んでリードを広げた。日進月歩の成長を見せる吉尾の今後の活躍に注目だ。
8,1,1,2-1,Nah
9,0,0,2-0,前半の大分は、相手にコンパクトな陣形を整えられると、ビルドアップに苦慮。ショートカウンターから危ない場面を招くなど、相手の術中にはまってしまう。後半開始直後に岩田が先制点を獲得するも、ペースは変わらず、逆に一段とギアの上がった仙台の猛反撃を受ける。この流れを変えるべく、オナイウを投入すると、前線の選手がボールに絡めるようになり、追加点を奪取。終盤では持ち前の運動量を生かして完封に成功。2連勝で3位に浮上した。一方の仙台は、前半の出来は良かっただけに決定力に泣いた試合となった。


,results,home,score,match review
0,1,0,4-1,Nah
1,0,1,1-2,GKを含めて最終ラインからの速攻を狙う大分に対し、サイドに追い込んで効率的にボールを奪う鹿島。攻撃面では細かいパスワークから空いた逆サイドにつないで幾度かチャンスを創出する。守備面では相手の速攻における中盤へのパスはレオシルバが対処したものの、最終ラインの裏を狙ったロングパスの処理がおろそかになってしまった。これで藤本やオナイウに前を向かれ、藤本には先制点と勝ち越し点を許す結果に。次節の相手は高い攻撃力を誇る川崎Fであるだけに、守備面の修正が必要になるだろう。
2,2,0,1-1,前半は川崎Fが主導権を握る。パスを細かくつないでボールを保持し、サイド、中央を問わずに好機を創出。特に、加入後初スタメンの馬渡は右サイドで多くのチャンスに関与し、スムーズな連係を披露した。両者1点を奪い合って後半に入ると、相手のカウンターでボールを自陣深くに持ち込まれる場面が増加。チョンソンリョンの冷静な対応とDF陣の踏ん張りでしのぐと、終了間際に決定機が訪れる。しかし、知念のシュートはクォンスンテの好守に阻まれてしまう。ともに目指したリーグ戦初勝利は、次節以降へ持ち越される結果となった。
3,1,0,2-1,Nah
4,1,1,1-0,序盤から両者とも守備の強度が高く、五分五分の攻防が続く。だが、前半の終了間際に湘南が退場者を出したことで状況は一変。鹿島が相手を押し込んでボールを回し続ける一方的な展開となる。すると、後半13分に相手を揺さぶって待望の先制点を奪う。その後はカウンターに警戒しながら幅を使って攻め込み、最後のパワープレーはシャットアウト。今季初のクリーンシートでリーグ戦初勝利を達成した。対する湘南は右サイドが機能していただけに、岡本の退場は痛恨。1人少ない状況でも懸命に走り続けたが、2連敗となってしまった。
5,2,0,2-2,Nah
6,1,0,1-3,清水戦では、鈴木とAロペスの前線2人で5得点を奪った札幌だが、今節の相手は11試合連続勝ちなしと相性の悪い鹿島。積極的な攻撃を展開するが、「試合巧者な鹿島」に2点リードを許し、試合を折り返す。後半開始から徐々に攻撃のリズムを取り戻し、押し込む時間帯が続くも、レアンドロに3点目を奪われてしまう。後半40分にAロペスの2試合連続ゴールで1点を返すが、反撃もここまで。今回も苦汁をなめさせられる結果となった。
7,2,0,1-1,不調に陥る磐田はホームでまたしても勝利を逃した。前半から高い集中力で球際に競り勝ち、相手を押し込む形を創出するが、肝心なフィニッシュの場面でミスが目立ち、得点力不足の影が顔をのぞかせ始める。それでも、後半立ち上がりに松本がこん身の一撃。サポーターの雰囲気は最高潮に達し、勝利への期待が一気に高まる。しかし、鹿島の圧力に押されて次第にセカンドボールが拾えなくなり、防戦一方の展開に。完璧な崩しから守備網を破られ、同点とされてタイムアップ。長いトンネルを抜け出すには、まだまだ時間がかかりそうだ。
8,1,1,2-1,ホームの鹿島は、序盤から名古屋にサイドを突かれて主導権を握られる苦しい展開に。無失点で試合を折り返すも、後半開始早々に先制点を献上。その後も局面を打開できずにいたが、選手交代を機に前線の動きが活性化する。後半27分に途中出場の安部が起点となって同点に追い付き、9分後にはレオシルバが力技で逆転弾をもぎ取る。劣勢をわずかなチャンスではね返し、勝負強さを見せる結果となった。一方の名古屋は、ビハインドを背負ってから前線に枚数をかけて反撃に出るも、流れを引き戻すことはできず。敵地で痛い敗戦となった。
9,1,0,2-3,Nah


,results,home,score,match review
0,0,0,1-0,川崎Fが「平成最後のタイトルマッチ」を制した。前半から持ち味である速いテンポのパス回しを披露して優位に進め、好機を創出。特に輝きを放ったのは、新戦力のLダミアンだ。適応力とアジリティーの高さを見せるストライカーは相手守備陣に難しい対応を迫り、後半7分には挨拶代わりの一撃を突き刺して先制弾を奪取。その後はペースを落とし、1点を守り切って勝利を収めた。一方、浦和は守備に追われて後手に回る時間が長く、90分を通して放ったシュートはわずか1本。点差以上の完敗を喫し、攻撃面で課題を残した。
1,2,0,0-0,互いに不完全燃焼のまま90分を終えた。仙台は右サイドを主軸に据えて攻め込む。蜂須賀とハモンロペスは敵陣で多くボールに触れ、長沢のフィニッシュシーンを演出するが、攻め上がる回数が少ないこともあってネットを揺らすまでには至らない。逆に相手のパスワークに振り回され、守備に追われる時間を長く過ごす展開に。それでも、集中力を切らさずに体を張り続け、無失点を達成したことは収穫か。一方、浦和はボールこそ運べるものの、枠をとらえる強いシュートは数少なく、いまだに「調整段階」であることを印象づける結果となった。
2,0,1,0-2,開始早々に出はなをくじかれた浦和。これで相手に勢いと自信を与えてしまった。ボールを持った際は札幌の素早いプレスに自由を奪われ、またチャナティップを中心とした華麗なパスワークで何度もゴールを脅かされる展開に。シュートミスや西川のセーブに救われて早い時間帯での大量失点は回避したものの、後半も大勢は変わらず。終盤は山中や途中出場のAナバウトの突破から波状攻撃を見せるも、攻撃のスイッチを入れるにはあまりにも遅すぎた。結果、なすすべもなく0－2で完敗し、かつての恩師に勝点3の手土産を渡してしまった。
3,1,0,3-0,Nah
4,1,0,0-1,4季ぶりにJ1開催となった「アルウィン」での一戦は、ホームチームに多くの課題と収穫を与えた。序盤から素早い切り替えと積極的なプレッシングを見せ、相手に決定機を作らせない。後半も、勢いは衰えずに主導権を握るも、一瞬の隙からPKを献上し、後半27分に失点。終盤、反撃を見せるも、阿部や鈴木の投入で守備を固めた相手の守備陣を崩せずに今季初黒星を喫した。一方の浦和は、本調子とは程遠いパフォーマンスでありながらも、しぶとく戦って「勝点3」を奪取。上位進出に向けて勝負強さに自信を付けた。
5,2,0,0-0,Nah
6,1,0,1-2,いや応なしに注目を集めた「桜の新旧9番対決」。「新9番」都倉は、力強さとテクニカルな足技を交えたプレーで起点となり続けた。その他のタレントも美しい連動を見せ、柿谷とソウザは息の合ったスイッチでゴールを脅かすも、流れから浦和の牙城を破るには至らず。理想的な攻撃を展開しながらも結果が追い付かない、もどかしい一戦に。すると、後半31分には興梠に。同37分には「桜の旧9番」杉本にも得点を許してしまう。試合はこのままタイムアップとなり、リーグ戦3連敗となってしまった。
7,2,1,1-1,浦和は敵陣でボールを回し、チャンスを作ろうとするが、自陣に全員が戻り、コンパクトな守備を披露するFC東京を攻略できない状況に苦心。好機が生まれず、堅いゲームが続いたが、後半30分にわずかなチャンスを生かしたアウェイチームの華麗なカウンター攻撃で得点を許してしまう。失点後もボールを保持するが、決定機を作れないジレンマに陥る。それでも、ラストプレーで山中の鋭いクロスに反応した森脇が冷静に押し込み、土壇場で同点弾を奪取。埼玉で04年から負けていない相性の良い相手から勝点1をもぎ取ることに成功した。
8,0,1,0-3,浦和は連係ミスから開始早々に先制点を献上すると、ディフェンスラインの裏を狙う攻撃に終始苦しめられ、何度もチャンスを作られる。しかし、守護神の西川がビッグセーブを連発し、追加点を許さずに前半を終える。ただ、後半もサイドをうまく使われる攻撃に4バックの新布陣では対応できず、流れは変わらない。受け身のまま時間は過ぎ、内容でも結果でも完敗と言える「金J」となってしまった。一方、90分を通してゲームを支配し、三度もネットを揺らした横浜FMはアウェイの浦和戦を3連勝とし、リーグ戦4試合ぶりの勝利を飾った。
9,0,1,0-1,Nah


,results,home,score,match review
0,2,0,0-0,鬼木監督のもとで3連覇を見据える「蒼黒」と、長谷川体制2年目にさらなる飛躍を期す「青赤」。ともにブラッシュアップしたスタイルを披露した結果、ポゼッションを保つ川崎Fに対してFC東京がカウンターを狙うという構図が90分を通して続く一戦に。その中でも、前半はコンパクトネスを保ったFC東京が、後半は縦パスでの崩しを増やした川崎Fが試合を優位に進める。久保の美しいFKや中村の林との1対1など、惜しい場面こそみられたものの、最後まで得点は生まれず。火花散る「カイマクラシコ」は、ドロー決着となった。
1,1,0,2-3,ホーム開幕2連勝を目指す湘南はセットプレーから先制に成功するも、その後はカウンターで猛威を振るうFC東京の勢いに押し込まれる。一気に中盤を制圧され、ミスにも付け込まれて2点のビハインドを負う苦しい展開に。それでも、後半10分に武富の2試合連続ゴールで息を吹き返すと、ここから流れは湘南に傾く。ピッチの幅を広く使って攻め立て、終盤は途中出場の指宿をターゲットにロングボールも放り込みつつ同点を目指す。だが、ホイッスル直前のビッグチャンスを逸して万事休す。相性の悪い相手との乱打戦を落とした。
2,0,0,2-1,Nah
3,1,1,2-0,FC東京がホーム開幕戦を終盤の猛攻で制した。序盤から素早い攻守の切り替えで主導権を握りかけたが、永井の負傷交代後は動きが停滞。鳥栖のロングスローに手を焼きながらも、決定力不足に助けられてスコアレスで折り返す。後半は数的優位に立ちながらも攻めあぐねたが、終了間際にDオリヴェイラ、ジャエルの圧力を生かして2点を奪取。粘る相手から強引に勝利をもぎ取った。対する鳥栖はまたしても無得点でリーグ戦3連敗。Iクエンカの投入で流れをつかもうとしたが、直後に退場者を出した点が悔やまれる結果となった。
4,0,0,2-1,Nah
5,1,1,1-0,開幕3連勝と波に乗る名古屋をホームに迎えたFC東京。攻撃時には永井が再三裏へ走り込んで相手のビルドアッパーを低い位置へ引き連れ、失敗に終わっても東や高萩が素早く立ちふさがってパスコースを消す。それでも突破されれば、すぐに低くブロックを組んでその外で「持たせる」展開に持ち込む。また、単独突破を得意とする選手を投入された後半は、同じく途中投入の大森らがしつこく食らい付くなど、相手の攻撃プランを極限まで無効化。こうして柔軟な対応を見せたFC東京が永井の古巣への恩返し弾を守り切り、大一番を見事に飾った。
6,2,0,1-1,浦和は敵陣でボールを回し、チャンスを作ろうとするが、自陣に全員が戻り、コンパクトな守備を披露するFC東京を攻略できない状況に苦心。好機が生まれず、堅いゲームが続いたが、後半30分にわずかなチャンスを生かしたアウェイチームの華麗なカウンター攻撃で得点を許してしまう。失点後もボールを保持するが、決定機を作れないジレンマに陥る。それでも、ラストプレーで山中の鋭いクロスに反応した森脇が冷静に押し込み、土壇場で同点弾を奪取。埼玉で04年から負けていない相性の良い相手から勝点1をもぎ取ることに成功した。
7,1,1,2-1,「青赤」の勢いが止まらない。FC東京は前半から水際で粘る清水に屈する時間が続くと、後半早々に北川に先制点を奪われてしまう。ピッチには不穏な空気が流れたが、途中から出場した2人の新加入アタッカーがこのムードを払拭。ナサンホはJリーグ初ゴールを挙げ、ジャエルは絶妙なラストパスでDオリヴェイラの逆転弾を演出する。ここ一番で勝負強さを発揮し、ホーム3連勝を達成した。一方、敗れた清水も無敗のFC東京と遜色ない出来だったことは確か。だが、渇望する今季初白星を手にするには、最後の精度がわずかに足りなかった。
8,1,1,1-0,Nah
9,1,1,3-1,前半はFC東京のスタイルが見事に鹿島DF陣を崩壊させた。序盤から永井とDオリヴェイラのプレスや久保建の並はずれたテクニックでアウェイチームを翻弄。相手の動揺を誘い、前半29分までに流れの中から3ゴールを奪う。しかし、後半10分に1点を返されると、その後は防戦一方の試合運びに。試合終了まで劣勢を強いられたが、ここで運動量を落とさず徹底したプレッシングで粘り切った。上位対決を制し、開幕から7試合負けなしと好調をキープ。次節の天王山に向けて弾みがつく白星となった。


...

,results,home,score,match review
0,1,1,1-0,前半のC大阪は相手の「VIPトリオ」を中心としたボール回しへの対応に追われ、劣勢に立たされてしまう。それでも、新加入の奥埜を中心にパスコースを消す動きを続けて冷静に対応する。後半も同様の展開で進むが、途中出場の「新エース」が流れを変えた。都倉は最前線で堅実にポストプレーをこなし、後方からの味方の上がりを促す。先制点を生んだCKもこの働きから得たチャンスだった。リードを得た終盤はピッチ随所のスペースを「つぶす」配置で守りを固め、強力攻撃陣の完封に成功。夜のヤンマーに「桜が咲き」、春の訪れを告げた。
1,0,0,2-0,ホームの名古屋は、序盤から新加入の米本が起点となり、前線のジョーにボールを集めるものの、シュートに持ち込むシーンは少なく、スコアレスで試合を折り返す。後半に入ると、前田と赤崎を投入して現状の打破を図る。それでも相手のブロックを崩せずにいたが、後半34分に赤崎の得点で待望の先制点を獲得し、終了間際にも相手のミスを見逃さなかった赤崎が冷静にゴールに流し込んだ。ディフェンス面でも2試合連続のクリーンシートを達成し、2連勝を記録して最高のスタートを切ったといえるだろう。
2,0,0,2-1,Nah
3,0,1,0-1,ロティーナ監督の下、丁寧なビルドアップを試みるC大阪だったが、随所にミスが見られてピンチを招く苦しい展開。前半19分の失点も連係ミスから生まれ、その後はビハインドを喫した焦りからか敵陣での連動性を欠いてしまう。後半に入り、長短のパスを織り交ぜることで攻撃のバリエーションが生まれ始めるものの、ゴール前での精度を欠いて無得点で終了。公式戦3連敗となり、課題を残す結果となった。一方、広島は「ウノゼロ」での勝利。課題の得点力不足が解消したとは言い切れないものの、うれしい今季初白星を手にした。
4,2,0,0-0,Nah
5,0,1,1-2,いや応なしに注目を集めた「桜の新旧9番対決」。「新9番」都倉は、力強さとテクニカルな足技を交えたプレーで起点となり続けた。その他のタレントも美しい連動を見せ、柿谷とソウザは息の合ったスイッチでゴールを脅かすも、流れから浦和の牙城を破るには至らず。理想的な攻撃を展開しながらも結果が追い付かない、もどかしい一戦に。すると、後半31分には興梠に。同37分には「桜の旧9番」杉本にも得点を許してしまう。試合はこのままタイムアップとなり、リーグ戦3連敗となってしまった。
6,1,0,0-2,3連敗中のチーム同士の対戦は戦術の浸透度が勝負を決めた。開始直後こそ前線の3枚が掛けるプレッシングがはまった仙台だったが、徐々に相手の優れたパスワークに対して後手に回るようになり、中盤を制圧されてしまう。後半からは2シャドーのハモンロペスと吉尾のポジションを左右で入れ替えたが、コンビネーションプレーはあまり見られなかった。一方のC大阪は時間がたつにつれて選手の距離感が良くなり、奥埜とソウザが攻守に渡って奮闘。また、今季初出場の田中にゴールが生まれ、雪が降りしきるユアスタで快勝を収めた。
7,2,0,1-1,今季初の連勝を狙うホームの川崎Fは、序盤から敵陣にボールを運んで攻撃を組み立てるも、シュートに持ち込むシーンは少ない。すると、前半22分に柿谷にミドルシュートを決められて先制点を献上。それでも、後半の立ち上がりに知念の2試合連続ゴールで同点に追い付く。その後は攻撃的な選手を投入して猛攻を仕掛けるものの、逆転弾は生まれず。勝点1を分け合う結果となった。一方のC大阪は追い付かれてしまったものの、敵地2連戦で勝点4を獲得したことはポジティブにとらえられるだろう。
8,1,1,3-0,Nah
9,0,1,0-1,両チームともに締まったプレーを見せ、緊張感のある固い内容の試合となる。ホームのC大阪は連係でペナルティエリアへの進入を試みる札幌の攻撃に耐え、仕掛ける際には前への意識が高い片山や松田を中心に右サイドからの打開を狙う。後半に入ってもなお、一進一退の攻防が続くが、セットプレーから進藤に今季初得点を決められ、苦しい展開になる。その後は収まりの良い都倉に縦パスを入れて攻撃を仕掛け続けるも、タイムアップ。復調のけはいを見せていただけに、痛い敗戦を喫してしまった。


,results,home,score,match review
0,0,0,1-0,前半のC大阪は相手の「VIPトリオ」を中心としたボール回しへの対応に追われ、劣勢に立たされてしまう。それでも、新加入の奥埜を中心にパスコースを消す動きを続けて冷静に対応する。後半も同様の展開で進むが、途中出場の「新エース」が流れを変えた。都倉は最前線で堅実にポストプレーをこなし、後方からの味方の上がりを促す。先制点を生んだCKもこの働きから得たチャンスだった。リードを得た終盤はピッチ随所のスペースを「つぶす」配置で守りを固め、強力攻撃陣の完封に成功。夜のヤンマーに「桜が咲き」、春の訪れを告げた。
1,1,1,1-0,「脅威的なDF」が来日した。加入後即スタメン出場を飾ったダンクレーは効果的なフィードを供給し、何度もチャンスを演出。神戸が掲げるポゼッションサッカーに最適であることをアピールする。また、守備面でもFトーレスらを封殺するなど躍動した。そして、注目の一戦の勝敗を決めたのはダビドビジャ。ワンチャンスを確実に決めてスターの片りんを見せつけた。Aイニエスタは観客を沸かせるさすがのテクニックを幾度も披露。スター軍団の躍進が期待されるゲームとなった。一方の鳥栖は2連敗。攻撃陣の奮起が待たれる。
2,2,0,2-2,Nah
3,1,0,1-3,「特別な一戦で勝利を」という大きな決意を胸に臨んだホームの仙台。手数を掛けない高速カウンターで何度もゴールに迫り、リードを奪った立ち上がりの試合運びにその強い意気込みを感じたサポーターも多いだろう。だが、前半のうちに追い付かれ、さらに後半早々に出はなをくじかれた後は中盤で安易なミスを連発。手渡されたチャンスを「VIPトリオ」が逃すはずもなく、結果的に3失点を喫して敗戦。石原崇や吉尾ら新加入選手が存在感を発揮した点は収穫だが、守備面で大きな課題を残し、リーグ戦初白星は次節以降に持ち越しとなった。
4,2,1,0-0,Nah
5,2,1,1-1,3連勝を目指す神戸はホームに今季リーグ戦未勝利の17位清水を迎えた。神戸は前半、主導権を握るものの、清水の激しいプレスの前に効果的な攻撃を繰り出せない。だが、後半は「VIPトリオ」を中心として幾度かチャンスを創出。すると、後半4分にライン際で粘ったAイニエスタのクロスにDFの処理ミスを見逃さなかったLポドルスキがゴールを決め、「四股踏みパフォーマンス」を披露し、観客を沸かした。しかし、試合終了間際に一瞬の隙を突かれて同点にされる。勝利を手中に収めていただけに、神戸にとっては悔しい結果となった。
6,1,0,3-4,37076人とパナスタ史上最多入場者数を記録した「関西ダービー」に敗れ、G大阪はまたもホームで白星を挙げられなかった。早い段階で2点をリードするも、「VIPトリオ」の連係からあっさりと同点に追い付かれる。ゲーム終盤には疲れから足が止まりだしたところを「伏兵」田中に突かれて万事休す。「チームの心臓」である遠藤が要所でらしいプレーを見せるなど、攻撃は厚みがあっただけに守備の再編が急務となる。一方の神戸は、劣勢から劇的な勝利。カウンター攻撃への対応は課題だが、圧倒的な攻撃力を見せつけた。
7,0,0,2-1,松本が今季のホーム初勝利を挙げるとともに、連敗を3でストップした。松本は連動した守備と縦に速い攻撃で開始からリズムをつかむと、セットプレーから2点を先行する。後半は神戸の勢いに押される展開となり、高さのあるウェリントンをターゲットにしたサイド攻撃に苦しめられ、1点を返されてしまう。その後も流れは変わらなかったが、3バックを中心とする守備陣が踏ん張り、リードを守り切った。一方、神戸は反撃が及ばず、開幕戦以来の敗北を喫した。
8,1,1,2-0,Nah
9,0,1,2-4,神戸は前線から連動したプレスで相手のミスを誘発すると同時に、テンポの良いパスワークで攻撃を展開。効果的にゴールへと迫り、チャンスを確実に仕留めて先制点を奪い、一時は追い付かれるもすぐに勝ち越して折り返す。ただ、後半は逆転を狙う相手の勢いにのまれると、サイドからの攻撃に対応できず、立て続けに3失点。2－4で終了し、手痛い逆転負けを喫した。一方、広島は集中力を欠いてセットプレーから2点を許したものの、渡のアクロバティックなゴールなどで後半に見事に盛り返す。粘り強さと大胆さで首位をキープした。


,results,home,score,match review
0,1,0,0-0,Nah
1,2,1,1-1,1－1の引き分けに終わった一戦は、開幕節にふさわしい好ゲームとなった。先制点を奪ったのは、アウェイの清水だ。前半30分に新加入の中村、金子とのコンビネーションから北川がゴールを決め、「1トップ2シャドー」がいきなり結果を残す。一方、前半こそ苦しんだ広島であったが、後半10分にパトリックを投入すると、流れが一変。同12分、パトリックを狙ったクロスが起点となり、新加入のEサロモンソンが同点弾を奪う。両チームともに新加入選手が得点に関わり、新たなピースが大きく貢献したゲームとなった。
2,2,1,0-0,ホーム2連戦となった広島は序盤から磐田の「サイドバックが中に入り、ボールを受ける」という特徴的なビルドアップにうまく対処するも、守勢に回る時間が長くなってしまう。ただ、守備陣はフィニッシュの場面で思うようにさせず、高い集中力を保ちながら堅実なプレーを披露した。一方の攻撃面では縦への意識を高く持ち、パトリックの身体能力の高さを生かして攻め込む。後半は徐々にペースをつかみ、サイドからの攻撃で何度か決定機を作るも、磐田の堅い守りを崩せず、スコアレスドローで決着。強固な守備をともに披露した一戦となった。
3,0,0,2-0,Nah
4,1,0,0-1,ロティーナ監督の下、丁寧なビルドアップを試みるC大阪だったが、随所にミスが見られてピンチを招く苦しい展開。前半19分の失点も連係ミスから生まれ、その後はビハインドを喫した焦りからか敵陣での連動性を欠いてしまう。後半に入り、長短のパスを織り交ぜることで攻撃のバリエーションが生まれ始めるものの、ゴール前での精度を欠いて無得点で終了。公式戦3連敗となり、課題を残す結果となった。一方、広島は「ウノゼロ」での勝利。課題の得点力不足が解消したとは言い切れないものの、うれしい今季初白星を手にした。
5,1,0,2-1,Nah
6,1,1,1-0,松本をEスタに迎えた広島が、攻守において試合を支配した。守備では、吉野を中心としたディフェンスラインで松本の「スピードスター」前田を抑え込む。一方の攻撃では、柏とEサロモンソンのサイド攻撃を軸にチャンスを創出。前半で得点を奪えなかったものの、後半に歓喜の時が訪れた。後半16分に柏がペナルティエリア手前でDFに囲まれながらも、狙い澄ましたシュートを決めて先制に成功。その後も主導権を手放さず、終了間際のパワープレーもしのいで勝点3をゲット。同時にJ1ホーム通算200勝を達成した。
7,1,0,0-1,大分は前線の選手が果敢に裏への抜け出しを狙うが、広島の強固な守備に対して効果的な縦パスを入れられず。一方の守備陣は、ミスからピンチを招く場面もあり、ボールを持ちながらも主導権を握れない展開となる。後半12分に先制点を献上すると、その後は一層守備の意識が強くなった相手のディフェンスを前に、ホームチームは中盤でボールを回し続けて終了の笛を聞くこととなった。一方でアウェイチームは、佐々木を中心に城壁を築き、リーグ戦4試合連続の無失点と好調を維持。Dヴィエイラの加入後初ゴールが値千金の決勝弾となった。
8,1,1,3-0,ホームの広島が完璧な試合運びで、危なげなく勝点3を奪った。90分間を通して前線から積極的にプレスを掛け、G大阪の分厚い中盤に自由を与えない。ボールを奪った際には、2シャドーの連係を生かしたカウンター攻撃を徹底。このやり方が見事にはまり、着実に得点を重ねる。リードを奪うと、チーム全体でタイトな守備ブロックを築き上げ、強力なアウェイチームの攻撃陣を封殺。今季5回目のクリーンシートを達成し、攻守で充実したパフォーマンスをサポーターに披露した。
9,1,1,2-0,Nah


,results,home,score,match review
0,0,1,0-4,鳥栖は終始、名古屋のポゼッションに対してリトリートでしのぐゲーム展開となる。立ち上がりからややファウルが多いものの、球際で強さを発揮し、最終ラインでしっかりとブロックを構築することでシャットアウト。攻撃に転じればFトーレスを中心に何度かチャンスを作った。後半もこの流れのままゲームを進めていたが、ジョーに一瞬の隙を突かれて失点を喫すると、ここから牙城が崩される。名古屋のテンポの良い攻撃に対し、バラバラに乱れた守備では対応できず、最終的には4失点。ルイスカレーラス新体制の船出は厳しいものとなった。
1,0,0,1-0,「脅威的なDF」が来日した。加入後即スタメン出場を飾ったダンクレーは効果的なフィードを供給し、何度もチャンスを演出。神戸が掲げるポゼッションサッカーに最適であることをアピールする。また、守備面でもFトーレスらを封殺するなど躍動した。そして、注目の一戦の勝敗を決めたのはダビドビジャ。ワンチャンスを確実に決めてスターの片りんを見せつけた。Aイニエスタは観客を沸かせるさすがのテクニックを幾度も披露。スター軍団の躍進が期待されるゲームとなった。一方の鳥栖は2連敗。攻撃陣の奮起が待たれる。
2,0,1,1-3,Nah
3,0,0,2-0,FC東京がホーム開幕戦を終盤の猛攻で制した。序盤から素早い攻守の切り替えで主導権を握りかけたが、永井の負傷交代後は動きが停滞。鳥栖のロングスローに手を焼きながらも、決定力不足に助けられてスコアレスで折り返す。後半は数的優位に立ちながらも攻めあぐねたが、終了間際にDオリヴェイラ、ジャエルの圧力を生かして2点を奪取。粘る相手から強引に勝利をもぎ取った。対する鳥栖はまたしても無得点でリーグ戦3連敗。Iクエンカの投入で流れをつかもうとしたが、直後に退場者を出した点が悔やまれる結果となった。
4,1,0,0-1,Nah
5,1,1,1-0,勝点3を渇望する両者の対決は劇的な幕切れが待っていた。序盤から攻守の切り替えを徹底し、前半をスコアレスで終えた鳥栖。だが、後半開始直後にFトーレスが負傷交代を余儀なくされ、さらにその後には2試合連続となる退場者を出してしまう。劣勢に立たされたものの、ホームチームはサポーターの声援を受け、決して集中を切らさずに相手に押し込まれる時間帯を耐えしのいだ。すると、途中投入されたIクエンカが終了間際に値千金のヘディングシュートをたたき込んで試合終了。待望の今季初勝利に駅スタは歓喜に沸いた。
6,2,0,0-0,横浜FMは序盤から仲川や三好らを中心に右サイドから攻め込み、今季初出場となった松原の積極的な攻撃参加が良いアクセントとなってゴールへ迫りながら前半を終えた。後半に入っても流れは変わらず。仲川が後半18分に退くも、代わりに入った遠藤が勢いを落とすことなく3トップの攻撃陣が果敢に攻め立てる。だが、最後まで相手の牙城を崩せずに引き分けに終わった。一方の鳥栖は依然として得点力不足に悩む結果に。前節に続いての白星獲得とはならなかった。
7,0,0,3-0,横浜FMから加入した若武者がユアスタに今季初めての歓喜をもたらした。2戦連続の先発出場を果たした吉尾は、序盤から積極的な姿勢で攻撃に関与する。まず、輝きを放ったのは前半15分。巧みな動き出しで味方からのパスを引き出すと、右サイドから左足で正確なクロスを蹴り込み、ジャーメインの先制点をお膳立て。豊富な運動量でピッチを駆け回るレフティーは、後半9分にも得点に絡む。相手のパスミスに反応してシュートを放ち、ポストの跳ね返りを兵藤が押し込んでリードを広げた。日進月歩の成長を見せる吉尾の今後の活躍に注目だ。
8,0,0,1-0,Nah
9,0,1,0-1,前半は鳥栖が主導権を握って優位に進める。相手にボールが渡ると、ラインを高く設定し、隙の少ないコンパクトなブロックを構築。的確なタイミングで前線からプレスを掛け、川崎Fが得意とするパスワークを封じる。攻勢に転じると、金崎を中心とした攻撃陣は果敢に仕掛けて好機を創出した。しかし、後半6分に先制を許すと、緩急を駆使してペースを落とす相手の術中にはまって時計の針を進められる。攻撃的な選手を投入するも、状況は好転せずにタイムアップ。今季6試合目となる無得点試合で敗戦を喫した。


,results,home,score,match review
0,1,0,1-2,GKを含めて最終ラインからの速攻を狙う大分に対し、サイドに追い込んで効率的にボールを奪う鹿島。攻撃面では細かいパスワークから空いた逆サイドにつないで幾度かチャンスを創出する。守備面では相手の速攻における中盤へのパスはレオシルバが対処したものの、最終ラインの裏を狙ったロングパスの処理がおろそかになってしまった。これで藤本やオナイウに前を向かれ、藤本には先制点と勝ち越し点を許す結果に。次節の相手は高い攻撃力を誇る川崎Fであるだけに、守備面の修正が必要になるだろう。
1,0,1,0-1,戦前の予想通り、ボールの保持率では大分が圧倒。しかし、前線からのプレスに遭ってパスコースは限定され、高い位置に持ち込んでも相手のダブルボランチのケアを受けてほとんど決定機を作れない。後半早々の失点は不運な面も否めないが、前節2ゴールを決めた藤本にパスが入る場面も限られ、不完全燃焼のまま黒星を喫した。対する松本は先制点で永井がこぼれ球に誰よりも早く反応したように、足を止めなかったことが攻守両面で結実。アウェイの地まで駆け付けたサポーターと、約3年半ぶりとなるJ1勝利の味をかみ締めた。
2,1,1,2-1,Nah
3,1,0,1-2,退場者の有無が勝敗を分けることになった。前半、磐田は空いたスペースを突いてくる大分のカウンター攻撃に手を焼くことに。前半13分にそのカウンターから先制されると、同30分にはカウンターを止めようとした大南が藤本を倒してしまい、一発退場。早い時間帯で数的不利になってしまった。一時はアダイウトンがアクロバティックなシュートを決めて同点に追い付き、勢いづくが、後半に入ると大分に両サイドを揺さぶられて失点。その後も1人多い大分がボールを握り、大分優勢の状況を崩せず、タイムアップを迎えた。
4,0,0,2-1,Nah
5,1,1,2-0,しっかりとパスをつないで攻撃へと転じるビルドアップが特徴的なチーム同士の対戦。大分は前線からのプレスを受けながらも、相手の高いディフェンスラインの裏を後藤と藤本が狙い続けてチャンスを作る。すると、後半10分に藤本が先制点を挙げ、勢いのままに藤本はトリッキーなシュートを決めて追加点を奪う。その後は相手の怒とうの攻撃を集中力と粘り強さを見せてしのぎ、2－0のまま終了。09年以来となるホーム開催でのJ1勝利を達成した。
6,0,1,0-1,大分は前線の選手が果敢に裏への抜け出しを狙うが、広島の強固な守備に対して効果的な縦パスを入れられず。一方の守備陣は、ミスからピンチを招く場面もあり、ボールを持ちながらも主導権を握れない展開となる。後半12分に先制点を献上すると、その後は一層守備の意識が強くなった相手のディフェンスを前に、ホームチームは中盤でボールを回し続けて終了の笛を聞くこととなった。一方でアウェイチームは、佐々木を中心に城壁を築き、リーグ戦4試合連続の無失点と好調を維持。Dヴィエイラの加入後初ゴールが値千金の決勝弾となった。
7,1,0,1-2,前からの激しいプレスと多くの味方の攻撃参加でゴールを目指す札幌と、後方からつなぎつつ、有効なスペースを突いて攻め込む大分。両者のスタイルのぶつかり合いが終始続いた一戦となった。札幌は開始早々に先手を取られると、プレスの意識をさらに強化。だが、これで裏のスペースを空けると、次第に藤本らに突かれて2点目を献上する。大分としては狙い通りの、札幌としては勢いに水を差される1点となった。後半も試合の大勢はほぼ変わらず。ホームチームは90分にわたって攻撃に圧力を見せ続けただけに、悔しい敗戦となった。
8,0,0,2-0,Nah
9,1,1,2-0,前半の大分は、相手にコンパクトな陣形を整えられると、ビルドアップに苦慮。ショートカウンターから危ない場面を招くなど、相手の術中にはまってしまう。後半開始直後に岩田が先制点を獲得するも、ペースは変わらず、逆に一段とギアの上がった仙台の猛反撃を受ける。この流れを変えるべく、オナイウを投入すると、前線の選手がボールに絡めるようになり、追加点を奪取。終盤では持ち前の運動量を生かして完封に成功。2連勝で3位に浮上した。一方の仙台は、前半の出来は良かっただけに決定力に泣いた試合となった。
